Uses the method to fit a Log Pearson Type III distribution to the flood peaks detailed at:

https://streamflow.engr.oregonstate.edu/analysis/floodfreq/index.htm


In [1]:
import numpy as np
import pandas as pd
import datetime

In [2]:
input_data='./input_data/'
output_data='./output_data/'

In [3]:
yearly = pd.read_csv(output_data + 'chavuma_yearly.csv')
yearly.head(3)

,WaterYear,Flow_min,Flow_median,Flow_mean,Flow_max,Flow_range
0,1959,38.0,262.0,814.530812,5477.0,5439.0
1,1960,55.0,287.0,1048.950685,5480.0,5425.0
2,1961,77.0,432.0,1123.441096,5725.0,5648.0


In [4]:
yearly=yearly.sort_values('Flow_max',ascending=False)
yearly['rank'] = np.arange(len(yearly))+1
n=len(yearly)
yearly.head(3)

,WaterYear,Flow_min,Flow_median,Flow_mean,Flow_max,Flow_range,rank
9,1968,78.0,392.000000,1386.516438,7416.0,7338.0,1
8,1967,59.0,503.825581,1474.021858,6035.0,5976.0,2
19,1978,77.0,378.000000,964.052055,5735.0,5658.0,3


In [5]:
yearly['log_q']=np.log10(yearly['Flow_max'])
av_log_q=yearly['log_q'].mean()
yearly['log_q_m2']=(yearly['log_q']-av_log_q)**2
yearly['log_q_m3']=(yearly['log_q']-av_log_q)**3
yearly['ret']=(n+1)/yearly['rank']
yearly['exceed']=1/yearly['ret']

yearly.head(2)

,WaterYear,Flow_min,Flow_median,Flow_mean,Flow_max,Flow_range,rank,log_q,log_q_m2,log_q_m3,ret,exceed
9,1968,78.0,392.000000,1386.516438,7416.0,7338.0,1,3.870170,0.183108,0.078354,61.0,0.016393
8,1967,59.0,503.825581,1474.021858,6035.0,5976.0,2,3.780677,0.114527,0.038758,30.5,0.032787


In [6]:
sum_m2=yearly['log_q_m2'].sum()
sum_m3=yearly['log_q_m3'].sum()

var=sum_m2/(n-1)
sd=var**0.5
skew=n*sum_m3/((n-1)*(n-2)*sd**3)
(var,sd,skew)

(0.0635975124566041, 0.25218547233455796, -0.6032097085748844)

In [30]:
skew_coef=pd.read_csv(input_data + 'skew_coefficients.csv').set_index('Cs')
skew_coef

,1.0101,2,5,10,25,50,100,200
Cs,,,,,,,,
3.0,-0.667,-0.396,0.420,1.180,2.278,3.152,4.051,4.970
2.9,-0.690,-0.390,0.440,1.195,2.277,3.134,4.013,4.904
2.8,-0.714,-0.384,0.460,1.210,2.275,3.114,3.973,4.847
2.7,-0.740,-0.376,0.479,1.224,2.272,3.093,3.932,4.783
2.6,-0.769,-0.368,0.499,1.238,2.267,3.071,3.889,4.718
...,...,...,...,...,...,...,...,...
-2.6,-3.899,0.368,0.696,0.747,0.764,0.768,0.769,0.769
-2.7,-3.932,0.376,0.681,0.724,0.738,0.740,0.740,0.741
-2.8,-3.973,0.384,0.666,0.702,0.712,0.714,0.714,0.714


In [33]:
skew_lower=np.floor(skew*10)/10
skew_upper=np.ceil(skew*10)/10
rets=pd.DataFrame([skew_coef.loc[skew_lower],skew_coef.loc[skew_upper]]).transpose()
rets['Slope']=rets[skew_lower]/rets[skew_upper]
rets['k']=rets[skew_upper]-((skew_upper-skew)*rets['Slope'])
rets['Q']=10**(av_log_q+(rets['k']*sd))
rets.index.name='Return'
rets=rets['Q']
rets

Return
1.0101     558.031059
2         2926.017958
5         4545.404843
10        5547.325561
25        6711.355004
50        7503.008829
100       8233.607679
200       8910.286502
Name: Q, dtype: float64

In [34]:
rets.to_csv(output_data + 'chavuma_floodreturns.csv')